In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Daten laden
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

# Trainingsdaten aufteilen
X = train_data.iloc[:, 1:].values  # Pixelwerte
y = train_data.iloc[:, 0].values   # Labels

# Normalisierung der Pixelwerte
X = X / 255.0
X_test = test_data.values / 255.0

# Reshape für CNN (28x28 Bilder mit 1 Kanal)
X = X.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Labels in One-Hot-Encoding umwandeln
y = to_categorical(y, num_classes=10)

# Daten aufteilen in Training und Validierung
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Datenaugmentation vorbereiten
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(X_train)

# CNN-Modell erstellen
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Modell kompilieren
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early Stopping, um Overfitting zu vermeiden
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modell trainieren mit Datenaugmentation
history = model.fit(datagen.flow(X_train, y_train, batch_size=64),
                    validation_data=(X_val, y_val),
                    epochs=50,
                    callbacks=[early_stopping])

# Vorhersagen für den Testdatensatz
predictions = np.argmax(model.predict(X_test), axis=1)

# Submission-Datei erstellen
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(predictions) + 1),
    "Label": predictions
})



Epoch 1/50


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


525/525 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.7160 - loss: 0.9071 - val_accuracy: 0.9580 - val_loss: 0.1457
Epoch 2/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.9442 - loss: 0.1817 - val_accuracy: 0.9826 - val_loss: 0.0572
Epoch 3/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.9576 - loss: 0.1465 - val_accuracy: 0.9840 - val_loss: 0.0543
Epoch 4/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9660 - loss: 0.1150 - val_accuracy: 0.9877 - val_loss: 0.0403
Epoch 5/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9679 - loss: 0.1056 - val_accuracy: 0.9869 - val_loss: 0.0440
Epoch 6/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9720 - loss: 0.0938 - val_accuracy: 0.9901 - val_loss: 0.0308
Epoch 7/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9734 - loss: 0.0865 - val_accuracy: 0.9912 - val_loss: 0.0266
Epoch 8/50
525/525 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9752 - loss: 0.0855 - val_accurac

In [3]:
submission.to_csv("submission1.csv", index=False)

print("Submission-Datei wurde erstellt: submission.csv")

Submission-Datei wurde erstellt: submission.csv
